# Exercices : Intégrité des Messages

**Objectifs** :
- Attaquer des MAC mal conçus
- Comprendre les collisions de hash
- Tester la sécurité d'AEAD
- Simuler un padding oracle attack

In [ ]:
import hashlib
import hmac
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import numpy as np
from collections import Counter

## Exercice 1 : Attaque sur MAC = H(k || m)

**Contexte** : Un système utilise $\text{MAC}(m) = H(k \| m)$ (clé || message).

**Tâche** : Démontrez l'attaque par extension de longueur (length extension attack).

**Indice** : Pour SHA-256, connaître $H(k \| m)$ permet de calculer $H(k \| m \| m')$ sans connaître $k$ !

In [ ]:
def weak_mac_concat(key: bytes, message: bytes) -> bytes:
    """
    MAC VULNÉRABLE : H(k || m)
    
    ⚠️ NE JAMAIS utiliser cette construction !
    """
    return hashlib.sha256(key + message).digest()

def length_extension_attack_demo():
    """
    Démonstration de l'attaque par extension de longueur.
    
    Note : Cette attaque est conceptuelle. L'implémentation complète
    nécessite de manipuler l'état interne du hash, ce qui est complexe.
    """
    print("=" * 70)
    print("ATTAQUE : Length Extension sur H(k || m)")
    print("=" * 70)
    
    # Système légitime
    secret_key = secrets.token_bytes(16)  # Clé secrète inconnue d'Eve
    original_message = b"amount=100"
    
    # MAC original (connu d'Eve)
    original_mac = weak_mac_concat(secret_key, original_message)
    
    print(f"\n✅ Système légitime :")
    print(f"   Message : {original_message}")
    print(f"   MAC     : {original_mac.hex()[:32]}...")
    
    print(f"\n💀 Eve veut forger : m' = '{original_message.decode()}|amount=999'")
    print(f"\n⚠️  VULNÉRABILITÉ :")
    print(f"   SHA-256 utilise Merkle-Damgård construction")
    print(f"   H(k || m) révèle l'état interne après traitement de k||m")
    print(f"   → Eve peut calculer H(k || m || padding || m') sans connaître k !")
    
    print(f"\n✅ SOLUTION : Utiliser HMAC au lieu de H(k || m)")
    print(f"   HMAC = H((k ⊕ opad) || H((k ⊕ ipad) || m))")
    print(f"   → Double hachage empêche length extension")
    
    # Comparaison avec HMAC
    hmac_original = hmac.new(secret_key, original_message, hashlib.sha256).digest()
    
    print(f"\n📊 Comparaison :")
    print(f"   H(k||m)      : {original_mac.hex()[:32]}... ❌ VULNÉRABLE")
    print(f"   HMAC(k, m)   : {hmac_original.hex()[:32]}... ✅ SÉCURISÉ")

length_extension_attack_demo()

## Exercice 2 : Collision Birthday Attack

**Contexte** : Un système utilise un hash tronqué à 24 bits pour identifier des documents.

**Tâche** : Trouvez deux documents différents avec le même hash.

In [ ]:
def birthday_collision_attack(output_bits=24):
    """
    Attaque birthday pour trouver une collision.
    
    Args:
        output_bits: Taille du hash tronqué (24 bits par défaut)
    
    Returns:
        (doc1, doc2, hash_value, attempts)
    """
    print("=" * 70)
    print(f"BIRTHDAY ATTACK : Collision sur hash {output_bits}-bits")
    print("=" * 70)
    
    hash_table = {}
    attempts = 0
    
    # Complexité attendue : ~√(2^n) = 2^(n/2)
    expected = int(np.sqrt(2 ** output_bits) * 1.25)
    
    print(f"\nEspace de hash : 2^{output_bits} = {2**output_bits:,} valeurs")
    print(f"Essais attendus : ~{expected:,} (√(2^{output_bits}))")
    print(f"\n🔍 Recherche de collision...")
    
    while True:
        # Générer document aléatoire
        doc = secrets.token_bytes(32)
        
        # Hash tronqué
        h_full = hashlib.sha256(doc).digest()
        h_truncated = int.from_bytes(h_full, 'big') % (2 ** output_bits)
        
        attempts += 1
        
        if h_truncated in hash_table:
            # COLLISION TROUVÉE !
            doc1 = hash_table[h_truncated]
            doc2 = doc
            
            print(f"\n✅ COLLISION TROUVÉE après {attempts:,} essais !")
            print(f"\n📄 Document 1 : {doc1.hex()[:32]}...")
            print(f"📄 Document 2 : {doc2.hex()[:32]}...")
            print(f"\n🔗 Hash commun ({output_bits} bits) : {h_truncated:0{output_bits//4}x}")
            
            # Vérifier hashes complets différents
            h1_full = hashlib.sha256(doc1).hexdigest()
            h2_full = hashlib.sha256(doc2).hexdigest()
            
            print(f"\n🔍 Hashes SHA-256 complets (différents) :")
            print(f"   Doc1 : {h1_full[:32]}...")
            print(f"   Doc2 : {h2_full[:32]}...")
            
            print(f"\n📊 Rapport théorie/pratique :")
            print(f"   Attendu  : ~{expected:,} essais")
            print(f"   Réel     : {attempts:,} essais")
            print(f"   Ratio    : {attempts/expected:.2f}x")
            
            return doc1, doc2, h_truncated, attempts
        
        hash_table[h_truncated] = doc
        
        # Progress (tous les 1000 essais)
        if attempts % 1000 == 0:
            print(f"   {attempts:,} essais...", end='\r')

# Test avec 20 bits (rapide)
doc1, doc2, h, attempts = birthday_collision_attack(output_bits=20)

print(f"\n⚠️  LEÇON : Hash tronqué = VULNÉRABLE")
print(f"   20 bits : collision en ~{attempts:,} essais (faisable)")
print(f"   64 bits : collision en ~2^32 essais (faisable avec GPU)")
print(f"   128 bits : collision en ~2^64 essais (difficile)")
print(f"   256 bits : collision en ~2^128 essais (infaisable)")
print(f"\n✅ C'est pourquoi SHA-256 utilise 256 bits (sécurité de 128 bits) !")

## Exercice 3 : Padding Oracle Attack (Simplifié)

**Contexte** : Un serveur déchiffre des messages AES-CBC et révèle si le padding est valide.

**Tâche** : Exploitez cet oracle pour déchiffrer un ciphertext sans connaître la clé !

**Note** : C'est une des attaques les plus célèbres en cryptographie (CVE-2014-0160, Heartbleed).

In [ ]:
class PaddingOracle:
    """
    Oracle vulnérable qui révèle si le padding est valide.
    
    ⚠️ NE JAMAIS implémenter ce comportement en production !
    """
    def __init__(self):
        self.key = secrets.token_bytes(16)
        self.queries = 0
    
    def encrypt(self, plaintext: bytes) -> tuple[bytes, bytes]:
        """
        Chiffre un message avec AES-CBC.
        
        Returns:
            (IV, ciphertext)
        """
        iv = secrets.token_bytes(16)
        
        # Padding PKCS7
        padder = padding.PKCS7(128).padder()
        padded = padder.update(plaintext) + padder.finalize()
        
        # Chiffrement CBC
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(padded) + encryptor.finalize()
        
        return iv, ciphertext
    
    def decrypt_and_check_padding(self, iv: bytes, ciphertext: bytes) -> bool:
        """
        Déchiffre et vérifie le padding.
        
        ⚠️ VULNÉRABILITÉ : Révèle si le padding est valide !
        
        Returns:
            True si padding valide, False sinon
        """
        self.queries += 1
        
        try:
            cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=default_backend())
            decryptor = cipher.decryptor()
            padded = decryptor.update(ciphertext) + decryptor.finalize()
            
            # Vérifier padding PKCS7
            unpadder = padding.PKCS7(128).unpadder()
            unpadder.update(padded) + unpadder.finalize()
            
            return True  # Padding valide
        except Exception:
            return False  # Padding invalide

def padding_oracle_attack_demo():
    """
    Démonstration simplifiée du padding oracle attack.
    
    On déchiffre le dernier byte d'un bloc sans connaître la clé !
    """
    print("=" * 70)
    print("PADDING ORACLE ATTACK (simplifié)")
    print("=" * 70)
    
    oracle = PaddingOracle()
    
    # Message secret
    secret_message = b"Secret message!"  # 15 bytes → padding \x01
    
    print(f"\n🔒 Message secret : {secret_message}")
    
    # Chiffrement
    iv, ciphertext = oracle.encrypt(secret_message)
    
    print(f"\n📡 Ciphertext intercepté :")
    print(f"   IV : {iv.hex()}")
    print(f"   CT : {ciphertext.hex()}")
    print(f"   Blocs : {len(ciphertext) // 16}")
    
    # Attaque sur le dernier byte du premier bloc
    print(f"\n🎯 Objectif : Déchiffrer le dernier byte sans connaître la clé")
    print(f"\n💡 Stratégie :")
    print(f"   1. Modifier IV[15] pour tester différentes valeurs")
    print(f"   2. Chercher IV[15]' tel que le padding soit \\x01")
    print(f"   3. Si padding valide : P[15] = IV[15] ⊕ IV[15]' ⊕ 0x01")
    
    print(f"\n🔍 Recherche par brute force (256 essais max) :")
    
    # Copier IV
    iv_modified = bytearray(iv)
    
    found = False
    for guess in range(256):
        # Modifier dernier byte de l'IV
        iv_modified[15] = guess
        
        # Tester avec l'oracle
        if oracle.decrypt_and_check_padding(bytes(iv_modified), ciphertext):
            # Padding valide !
            # Cela signifie que le dernier byte déchiffré est 0x01
            # P[15] = IV[15] ⊕ IV'[15] ⊕ 0x01
            
            plaintext_byte = iv[15] ^ guess ^ 0x01
            
            print(f"\n✅ TROUVÉ après {guess + 1} essais !")
            print(f"   IV'[15]        : 0x{guess:02x}")
            print(f"   P[15] récupéré : 0x{plaintext_byte:02x} = '{chr(plaintext_byte) if 32 <= plaintext_byte < 127 else "?"}'")
            
            # Vérification
            actual_last_byte = secret_message[-1] if len(secret_message) % 16 == 15 else 0x01
            print(f"   P[15] réel     : 0x{actual_last_byte:02x}")
            
            found = True
            break
    
    if not found:
        print(f"\n❌ Aucune valeur trouvée (cas particulier)")
    
    print(f"\n📊 Statistiques :")
    print(f"   Requêtes oracle : {oracle.queries}")
    print(f"   Bytes déchiffrés : 1/{len(ciphertext)}")
    print(f"\n💡 En répétant l'attaque, on déchiffre TOUT le message !")
    print(f"   Complexité : O(256 × longueur) requêtes")
    
    print(f"\n⚠️  VULNÉRABILITÉ : Révéler des erreurs de padding")
    print(f"✅ SOLUTION : Utiliser AEAD (AES-GCM) qui authentifie AVANT déchiffrement")

padding_oracle_attack_demo()

## Exercice 4 : AEAD Robustesse

**Contexte** : Tester la robustesse d'AES-GCM contre diverses modifications.

**Tâche** : Vérifiez qu'AES-GCM rejette toute modification du ciphertext, IV, ou associated data.

In [ ]:
def aead_robustness_tests():
    """
    Tests de robustesse d'AES-GCM.
    """
    print("=" * 70)
    print("TESTS DE ROBUSTESSE : AES-GCM")
    print("=" * 70)
    
    key = AESGCM.generate_key(bit_length=256)
    aesgcm = AESGCM(key)
    
    message = b"Sensitive data that must remain authentic"
    associated_data = b"User: Alice, Action: Transfer"
    nonce = secrets.token_bytes(12)
    
    # Chiffrement légitime
    ciphertext = aesgcm.encrypt(nonce, message, associated_data)
    
    print(f"\n✅ Chiffrement légitime :")
    print(f"   Message : {message}")
    print(f"   AD      : {associated_data}")
    print(f"   Nonce   : {nonce.hex()}")
    print(f"   CT      : {ciphertext.hex()[:32]}... ({len(ciphertext)} bytes)")
    
    # Test 1 : Modification d'un bit du ciphertext
    print(f"\n" + "=" * 70)
    print(f"TEST 1 : Modification d'un bit du ciphertext")
    print(f"=" * 70)
    
    modified_ct = bytearray(ciphertext)
    modified_ct[0] ^= 0x01  # Flipper 1 bit
    
    try:
        aesgcm.decrypt(nonce, bytes(modified_ct), associated_data)
        print(f"❌ ÉCHEC : Modification non détectée !")
    except Exception as e:
        print(f"✅ SUCCÈS : Modification détectée et rejetée")
        print(f"   Erreur : {type(e).__name__}")
    
    # Test 2 : Modification du nonce
    print(f"\n" + "=" * 70)
    print(f"TEST 2 : Modification du nonce")
    print(f"=" * 70)
    
    wrong_nonce = secrets.token_bytes(12)
    
    try:
        aesgcm.decrypt(wrong_nonce, ciphertext, associated_data)
        print(f"❌ ÉCHEC : Modification nonce non détectée !")
    except Exception as e:
        print(f"✅ SUCCÈS : Modification nonce détectée et rejetée")
        print(f"   Erreur : {type(e).__name__}")
    
    # Test 3 : Modification des associated data
    print(f"\n" + "=" * 70)
    print(f"TEST 3 : Modification des associated data")
    print(f"=" * 70)
    
    wrong_ad = b"User: Eve, Action: Transfer"  # Eve change le destinataire !
    
    try:
        aesgcm.decrypt(nonce, ciphertext, wrong_ad)
        print(f"❌ ÉCHEC : Modification AD non détectée !")
    except Exception as e:
        print(f"✅ SUCCÈS : Modification AD détectée et rejetée")
        print(f"   Erreur : {type(e).__name__}")
    
    # Test 4 : Troncature du ciphertext
    print(f"\n" + "=" * 70)
    print(f"TEST 4 : Troncature du ciphertext")
    print(f"=" * 70)
    
    truncated_ct = ciphertext[:-5]  # Supprimer 5 bytes
    
    try:
        aesgcm.decrypt(nonce, truncated_ct, associated_data)
        print(f"❌ ÉCHEC : Troncature non détectée !")
    except Exception as e:
        print(f"✅ SUCCÈS : Troncature détectée et rejetée")
        print(f"   Erreur : {type(e).__name__}")
    
    # Test 5 : Ajout de données
    print(f"\n" + "=" * 70)
    print(f"TEST 5 : Ajout de données au ciphertext")
    print(f"=" * 70)
    
    extended_ct = ciphertext + b"EXTRA"
    
    try:
        aesgcm.decrypt(nonce, extended_ct, associated_data)
        print(f"❌ ÉCHEC : Ajout non détecté !")
    except Exception as e:
        print(f"✅ SUCCÈS : Ajout détecté et rejeté")
        print(f"   Erreur : {type(e).__name__}")
    
    # Test 6 : Réutilisation (replay attack)
    print(f"\n" + "=" * 70)
    print(f"TEST 6 : Réutilisation du même message (replay attack)")
    print(f"=" * 70)
    
    try:
        # Déchiffrement initial
        plaintext1 = aesgcm.decrypt(nonce, ciphertext, associated_data)
        # Déchiffrement replay
        plaintext2 = aesgcm.decrypt(nonce, ciphertext, associated_data)
        
        print(f"⚠️  AES-GCM ne protège PAS contre replay attacks !")
        print(f"   Message déchiffré 2 fois : {plaintext1 == plaintext2}")
        print(f"\n💡 Protection contre replay : Utiliser un compteur de nonces côté serveur")
    except Exception as e:
        print(f"Erreur inattendue : {e}")
    
    print(f"\n" + "=" * 70)
    print(f"RÉSUMÉ")
    print(f"=" * 70)
    print(f"\n✅ AES-GCM détecte :")
    print(f"   - Modifications du ciphertext")
    print(f"   - Modifications du nonce")
    print(f"   - Modifications des associated data")
    print(f"   - Troncature ou extension")
    print(f"\n⚠️  AES-GCM NE détecte PAS :")
    print(f"   - Replay attacks (nécessite gestion de nonces)")
    print(f"   - Attaques hors bande (timing, side-channels)")

aead_robustness_tests()

## Exercice 5 : Comparaison Encrypt-then-MAC vs MAC-then-Encrypt

**Contexte** : Il existe trois façons de combiner chiffrement et MAC.

**Tâche** : Implémentez et comparez les trois approches.

In [ ]:
def encrypt_then_mac(message: bytes, key_enc: bytes, key_mac: bytes) -> tuple:
    """
    Encrypt-then-MAC (recommandé).
    
    c = Enc(m), t = MAC(c)
    Envoyer : (c, t)
    """
    # Chiffrement AES-CTR
    nonce = secrets.token_bytes(16)
    cipher = Cipher(algorithms.AES(key_enc), modes.CTR(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()
    
    # MAC du ciphertext
    tag = hmac.new(key_mac, nonce + ciphertext, hashlib.sha256).digest()
    
    return nonce, ciphertext, tag

def mac_then_encrypt(message: bytes, key_enc: bytes, key_mac: bytes) -> tuple:
    """
    MAC-then-Encrypt (vulnérable au padding oracle).
    
    t = MAC(m), c = Enc(m || t)
    Envoyer : c
    """
    # MAC du plaintext
    tag = hmac.new(key_mac, message, hashlib.sha256).digest()
    
    # Chiffrer message || tag
    nonce = secrets.token_bytes(16)
    cipher = Cipher(algorithms.AES(key_enc), modes.CTR(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message + tag) + encryptor.finalize()
    
    return nonce, ciphertext

def encrypt_and_mac(message: bytes, key_enc: bytes, key_mac: bytes) -> tuple:
    """
    Encrypt-and-MAC (utilisé par SSH, problématique).
    
    c = Enc(m), t = MAC(m)
    Envoyer : (c, t)
    """
    # Chiffrement
    nonce = secrets.token_bytes(16)
    cipher = Cipher(algorithms.AES(key_enc), modes.CTR(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()
    
    # MAC du plaintext (problématique !)
    tag = hmac.new(key_mac, message, hashlib.sha256).digest()
    
    return nonce, ciphertext, tag

# Comparaison
print("=" * 70)
print("COMPARAISON : Encrypt-then-MAC vs MAC-then-Encrypt vs Encrypt-and-MAC")
print("=" * 70)

key_enc = secrets.token_bytes(16)
key_mac = secrets.token_bytes(32)
message = b"Secret message to protect"

print(f"\nMessage : {message}\n")

# Encrypt-then-MAC
nonce1, ct1, tag1 = encrypt_then_mac(message, key_enc, key_mac)
print(f"1️⃣ Encrypt-then-MAC (TLS 1.3, IPsec) ✅ RECOMMANDÉ")
print(f"   Ordre : c = Enc(m), t = MAC(c)")
print(f"   CT    : {ct1.hex()[:32]}...")
print(f"   TAG   : {tag1.hex()[:32]}...")
print(f"   ✅ Vérifier MAC AVANT déchiffrement → Pas de padding oracle")

# MAC-then-Encrypt
nonce2, ct2 = mac_then_encrypt(message, key_enc, key_mac)
print(f"\n2️⃣ MAC-then-Encrypt (TLS 1.0-1.2) ⚠️  PROBLÉMATIQUE")
print(f"   Ordre : t = MAC(m), c = Enc(m || t)")
print(f"   CT    : {ct2.hex()[:32]}... (inclut le MAC)")
print(f"   ⚠️  Vulnérable au padding oracle attack (Lucky13, BEAST, POODLE)")

# Encrypt-and-MAC
nonce3, ct3, tag3 = encrypt_and_mac(message, key_enc, key_mac)
print(f"\n3️⃣ Encrypt-and-MAC (SSH) ⚠️  PROBLÉMATIQUE")
print(f"   Ordre : c = Enc(m), t = MAC(m)")
print(f"   CT    : {ct3.hex()[:32]}...")
print(f"   TAG   : {tag3.hex()[:32]}...")
print(f"   ⚠️  MAC du plaintext peut révéler des informations")

print(f"\n" + "=" * 70)
print(f"RECOMMANDATIONS")
print(f"=" * 70)
print(f"\n✅ Meilleure approche : AEAD (AES-GCM, ChaCha20-Poly1305)")
print(f"   → Chiffrement + authentification atomique")
print(f"   → Pas de risque d'implémentation incorrecte")
print(f"\n✅ Si AEAD indisponible : Encrypt-then-MAC")
print(f"   → Vérifier MAC AVANT déchiffrement")
print(f"   → Utiliser clés différentes pour Enc et MAC")
print(f"\n❌ Éviter : MAC-then-Encrypt et Encrypt-and-MAC")

## Conclusion

**Points clés vérifiés** :
- ❌ H(k || m) vulnérable à length extension → Utiliser HMAC
- ❌ Hash tronqué vulnérable à birthday attacks → Utiliser ≥256 bits
- ❌ Padding oracle permet de déchiffrer sans clé → Utiliser AEAD
- ✅ AES-GCM détecte toutes modifications (sauf replay)
- ✅ Encrypt-then-MAC est la bonne approche (ou AEAD)

**Retenir** :
- Toujours utiliser AEAD en production (AES-GCM, ChaCha20-Poly1305)
- Ne JAMAIS révéler d'erreurs de padding/déchiffrement
- Utiliser comparaisons constant-time pour MAC
- Gérer les nonces/compteurs pour éviter replay attacks